In [ ]:
#libraries
import pandas as pd
import spacy
from spacy import displacy
import en_core_web_sm
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import re
import string
import numpy as np
import random
import pandas as pd
from nltk.corpus import stopwords

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Loading spaCy's pre-trained model
nlp = en_core_web_sm.load()

# Loading the airline reviews dataset
df = pd.read_csv('Airline_Reviews.csv')

In [ ]:
df.head(5)

,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,NaN,Solo Leisure,Economy Class,Moroni to Moheli,November 2019,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,E120,Solo Leisure,Economy Class,Moroni to Anjouan,June 2019,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,Embraer E120,Solo Leisure,Economy Class,Anjouan to Dzaoudzi,June 2019,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no
3,3,Adria Airways,1,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,NaN,Solo Leisure,Economy Class,Frankfurt to Pristina,September 2019,1.0,1.0,NaN,1.0,NaN,NaN,1.0,no
4,4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,Couple Leisure,Economy Class,Sofia to Amsterdam via Ljubljana,September 2019,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no


In [ ]:
# Function to clean and preprocess a single tweet
def clean_and_preprocess(text):

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove mentions
    text = re.sub(r'@\w+', '', text)

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    return text

# Apply the cleaning function to each tweet in the 'text' column
df['cleaned_review'] = df['Review'].apply(clean_and_preprocess)

# Display the cleaned dataset
print("\nCleaned Dataset:")
print(df[ 'cleaned_review'].head())


Cleaned Dataset:
0      Moroni to Moheli Turned out to be a pretty d...
1     Moroni to Anjouan It is a very small airline ...
2      Anjouan to Dzaoudzi A very small airline and...
3      Please do a favor yourself and do not fly wi...
4     Do not book a flight with this airline My fri...
Name: cleaned_review, dtype: object


In [ ]:
df.head(5)

,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Aircraft,Type Of Traveller,Seat Type,...,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended,cleaned_review
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,NaN,Solo Leisure,Economy Class,...,November 2019,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes,Moroni to Moheli Turned out to be a pretty d...
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,E120,Solo Leisure,Economy Class,...,June 2019,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no,Moroni to Anjouan It is a very small airline ...
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,Embraer E120,Solo Leisure,Economy Class,...,June 2019,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no,Anjouan to Dzaoudzi A very small airline and...
3,3,Adria Airways,1,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,NaN,Solo Leisure,Economy Class,...,September 2019,1.0,1.0,NaN,1.0,NaN,NaN,1.0,no,Please do a favor yourself and do not fly wi...
4,4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,Couple Leisure,Economy Class,...,September 2019,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no,Do not book a flight with this airline My fri...


In [ ]:
 # Convert to lowercase
    text = text.lower()



In [ ]:
# Processing each review and extracting named entities
for i in range(0, 20):
    review = df['cleaned_review'][i]

# Process the review using spaCy NLP pipeline
    doc = nlp(review)

# Manually correct 'Ljubljana' to 'GPE'
    for token in doc:
        if token.text == 'Ljubljana':
            token.ent_type_ = 'GPE'

# Print entities and their labels
    print(f"Review #{i + 1}: {review}")
    for ent in doc.ents:
        print(f"Entity: {ent.text} - Label: {ent.label_}")

    print('\n')


Review #1:   Moroni to Moheli Turned out to be a pretty decent airline Online booking worked well checkin and boarding was fine and the plane looked well maintained Its a very short flight  just  minutes or so so i didnt expect much but they still managed to hand our a bottle of water and some biscuits which i though was very nice Both flights on time
Entity: Moheli Turned - Label: ORG
Entity: Online - Label: ORG
Entity: just  minutes or so - Label: TIME


Review #2:  Moroni to Anjouan It is a very small airline My ticket advised me to turn up at hrs which I did There was confusion at this small airport I was then directed to the office of AB Aviation which was still closed It opened at hrs and I was told that the flight had been put back to hrs and that they had tried to contact me This could not be true as they did not have my phone number I was with a local guide and he had not been informed either I presume that I was bumped off The later flight did operate but as usual there was c

In [ ]:
from collections import Counter
# Initializing a Counter to keep track of entity frequencies
entity_counter = Counter()

# Processing each review and extract named entities
for i in range(0,5000):
    review = df['Review'][i]
    doc = nlp(review)

    # Counting the frequency of each named entity
    for ent in doc.ents:
        entity_counter[ent.text] += 1

# Printing the most common named entities and their frequencies
print("Top 10 Most Common Named Entities:")
for entity, freq in entity_counter.most_common(10):
    print(f"Entity: {entity} - Frequency: {freq}")

Top 10 Most Common Named Entities:
Entity: first - Frequency: 733
Entity: one - Frequency: 502
Entity: 2 - Frequency: 454
Entity: two - Frequency: 358
Entity: 3 - Frequency: 315
Entity: London - Frequency: 242
Entity: First - Frequency: 222
Entity: Paris - Frequency: 214
Entity: second - Frequency: 209
Entity: Beijing - Frequency: 202
